# EAGLE3 Draft Model Evaluation - Interactive

This notebook walks through the evaluation process step-by-step so you can inspect:
- How prompts are formatted with chat templates
- How the loss mask is built
- What the draft model predicts vs target model
- Per-position accuracy and losses

Based on `scripts/eval_eagle.py`

## 1. Setup and Imports

In [ ]:
import os
import re
import time
from typing import Dict, List, Optional, Tuple

import torch
import torch.nn as nn
from transformers import AutoConfig, AutoTokenizer
from transformers.cache_utils import DynamicCache

from specforge import AutoDraftModelConfig, AutoEagle3DraftModel
from specforge.core.eagle3 import OnlineEagle3Model
from specforge.data.template import TEMPLATE_REGISTRY
from specforge.modeling.target import get_eagle3_target_model
from specforge.utils import padding

print("✅ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 2. Configuration

Set your paths and parameters here:

In [ ]:
# === CONFIGURATION ===
config = {
    "draft_checkpoint": "./outputs/eagle3-checkpoints/epoch_1_step_12528/",
    "target_model_path": "Qwen/Qwen2.5-7B-Instruct",
    "chat_template": "qwen",
    "num_samples": 3,  # Start with just 3 for interactive exploration
    "ttt_length": 7,  # Number of positions to predict ahead
    "max_length": 512,
    "attention_backend": "flex_attention",  # NOT flex_attention_mla for Qwen
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "target_model_backend": "hf",  # Use HuggingFace backend
    "vocab_mapping_path": None,  # Auto-detect
    "show_first_n_positions": 10,  # For detailed inspection
}

print("Configuration:")
for k, v in config.items():
    print(f"  {k}: {v}")

## 3. Test Data

These are simple prompts and responses for testing:

In [ ]:
TEST_PROMPTS = [
    "What is the capital of France?",
    "Explain quantum computing in simple terms.",
    "Write a Python function to calculate factorial.",
]

TEST_RESPONSES = [
    "The capital of France is Paris. Paris is located in the north-central part of the country and is the largest city in France.",
    "Quantum computing uses quantum bits or qubits that can exist in multiple states simultaneously, unlike classical bits that are either 0 or 1.",
    "Here's a Python function to calculate factorial:\n\ndef factorial(n):\n    if n <= 1:\n        return 1\n    return n * factorial(n - 1)",
]

num_samples = min(config["num_samples"], len(TEST_PROMPTS))
print(f"Using {num_samples} test samples")
for i in range(num_samples):
    print(f"\n{i+1}. Prompt: {TEST_PROMPTS[i]}")
    print(f"   Response: {TEST_RESPONSES[i][:80]}...")

## 4. Load Tokenizer and Chat Template

In [ ]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(config["target_model_path"])

print("\nTokenizer info:")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  BOS token: {tokenizer.bos_token} (id={tokenizer.bos_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (id={tokenizer.eos_token_id})")
print(f"  PAD token: {tokenizer.pad_token}")

# Load chat template
template = TEMPLATE_REGISTRY.get(config["chat_template"])
print(f"\nChat template ({config['chat_template']}):")
print(f"  User header: {repr(template.user_header)}")
print(f"  Assistant header: {repr(template.assistant_header)}")
print(f"  End of turn: {repr(template.end_of_turn_token)}")
print(f"  System prompt: {repr(template.system_prompt[:80] if template.system_prompt else None)}...")

## 5. Format a Single Sample (Inspect in Detail)

Let's look at how the first sample is formatted:

In [ ]:
# Take the first sample
sample_idx = 0
prompt = TEST_PROMPTS[sample_idx]
response = TEST_RESPONSES[sample_idx]

print(f"Sample {sample_idx + 1}:")
print(f"  Prompt: {prompt}")
print(f"  Response: {response}")

# Build messages
messages = [{"role": "user", "content": prompt}]
if template.system_prompt:
    messages = [{"role": "system", "content": template.system_prompt}] + messages
messages.append({"role": "assistant", "content": response})

print(f"\nMessages structure ({len(messages)} messages):")
for i, msg in enumerate(messages):
    content_preview = msg['content'][:60] + "..." if len(msg['content']) > 60 else msg['content']
    print(f"  {i+1}. {msg['role']}: {content_preview}")

# Apply chat template
try:
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    print("\n✅ Chat template applied successfully")
except Exception as e:
    print(f"\n❌ Error applying chat template: {e}")
    # Fallback
    text = f"{template.user_header}{prompt}{template.end_of_turn_token}{template.assistant_header}{response}{template.end_of_turn_token}"
    print("   Using fallback template")

print(f"\nFormatted text ({len(text)} chars):")
print("=" * 80)
print(text)
print("=" * 80)

## 6. Tokenize and Build Loss Mask

In [ ]:
# Tokenize
inputs = tokenizer(
    text, 
    return_tensors="pt", 
    return_offsets_mapping=True,
    max_length=config["max_length"], 
    truncation=True,
    add_special_tokens=False,  # Match training behavior
)

input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]
offsets = inputs["offset_mapping"][0]

print(f"Tokenization:")
print(f"  Input IDs shape: {input_ids.shape}")
print(f"  Total tokens: {input_ids.shape[1]}")

# Show first 20 tokens
print(f"\nFirst 20 tokens:")
for i in range(min(20, input_ids.shape[1])):
    token_id = input_ids[0, i].item()
    token_text = tokenizer.decode([token_id], skip_special_tokens=False)
    print(f"  {i:3d}: {token_id:6d} -> {repr(token_text)}")

In [ ]:
# Build loss mask (only assistant tokens count)
loss_mask = torch.zeros(input_ids.shape[1], dtype=torch.long)

user_sep = f"{template.end_of_turn_token or ''}{template.user_header}"
asst_sep = f"{template.end_of_turn_token or ''}{template.assistant_header}"
assistant_pattern = re.escape(asst_sep) + r"(.*?)(?=" + re.escape(user_sep) + "|$)"

print(f"Loss mask pattern:")
print(f"  User separator: {repr(user_sep)}")
print(f"  Assistant separator: {repr(asst_sep)}")
print(f"  Regex pattern: {assistant_pattern[:100]}...")

matches_found = 0
for match in re.finditer(assistant_pattern, text, re.DOTALL):
    matches_found += 1
    start_char = match.start(1)
    end_char = match.end(1)
    
    matched_text = text[start_char:end_char]
    print(f"\n  Match {matches_found}: chars [{start_char}:{end_char}]")
    print(f"    Content preview: {repr(matched_text[:80])}...")
    
    for idx, (tok_start, tok_end) in enumerate(offsets):
        if tok_end > start_char and tok_start <= end_char:
            loss_mask[idx] = 1

if matches_found == 0:
    print("\n⚠️  WARNING: No assistant spans found! Loss mask will be all zeros.")
else:
    assistant_tokens = loss_mask.sum().item()
    total_tokens = input_ids.shape[1]
    print(f"\n✅ Loss mask built: {assistant_tokens}/{total_tokens} tokens are assistant responses ({assistant_tokens/total_tokens:.1%})")

loss_mask = loss_mask.unsqueeze(0)  # Add batch dimension

In [ ]:
# Visualize loss mask
print("Token-by-token breakdown (first 30 tokens):")
print(f"{'Idx':<5} {'TokenID':<8} {'Token':<25} {'Mask':<5} {'Role'}")
print("-" * 70)

for i in range(min(30, input_ids.shape[1])):
    token_id = input_ids[0, i].item()
    token_text = tokenizer.decode([token_id], skip_special_tokens=False)
    mask_val = loss_mask[0, i].item()
    role = "ASST" if mask_val == 1 else "USER/SYS"
    
    # Truncate long tokens
    token_display = repr(token_text)[:23]
    print(f"{i:<5} {token_id:<8} {token_display:<25} {mask_val:<5} {role}")

## 7. Load Models

Now let's load the draft and target models:

In [ ]:
checkpoint_path = os.path.abspath(config["draft_checkpoint"])
if not os.path.isdir(checkpoint_path):
    raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

print(f"Loading draft model from: {checkpoint_path}")
draft_model = AutoEagle3DraftModel.from_pretrained(
    checkpoint_path,
    attention_backend=config["attention_backend"],
    torch_dtype=torch.bfloat16,
).to(config["device"]).eval()

draft_params = sum(p.numel() for p in draft_model.parameters())
print(f"   Parameters: {draft_params/1e6:.1f}M")
print(f"   Config: {draft_model.config}")

In [ ]:
# Load vocab mapping
vocab_mapping_path = config["vocab_mapping_path"]
if vocab_mapping_path is None:
    if os.path.exists(os.path.join(checkpoint_path, "vocab_mapping.pt")):
        vocab_mapping_path = os.path.join(checkpoint_path, "vocab_mapping.pt")
    else:
        # Try common cache locations
        for cache_dir in ["./cache/vocab_mapping", "../cache/vocab_mapping"]:
            if os.path.isdir(cache_dir):
                for f in os.listdir(cache_dir):
                    if f.endswith(".pt"):
                        vocab_mapping_path = os.path.join(cache_dir, f)
                        break
            if vocab_mapping_path:
                break

if vocab_mapping_path and os.path.exists(vocab_mapping_path):
    draft_model.load_vocab_mapping(vocab_mapping_path)
    print(f"✅ Loaded vocab mapping from: {vocab_mapping_path}")
else:
    print("⚠️  No vocab mapping found! Results may be inaccurate.")

In [ ]:
print(f"\nLoading target model: {config['target_model_path']}")
target_model = get_eagle3_target_model(
    pretrained_model_name_or_path=config["target_model_path"],
    backend=config["target_model_backend"],
    torch_dtype=torch.bfloat16,
    device=config["device"],
)

# Set aux hidden states layers
draft_config = draft_model.config
if (
    hasattr(draft_config, "eagle_config")
    and draft_config.eagle_config
    and "eagle_aux_hidden_state_layer_ids" in draft_config.eagle_config
):
    aux_layers = draft_config.eagle_config["eagle_aux_hidden_state_layer_ids"]
    target_model.set_aux_hidden_states_layers(aux_layers)
    print(f"   Aux hidden state layers: {aux_layers}")
else:
    target_model.set_aux_hidden_states_layers()
    print("   Using default aux hidden state layers")

# Estimate target params
target_config = AutoConfig.from_pretrained(config["target_model_path"])
target_params = getattr(target_config, "num_parameters", None)
if target_params is None:
    h = target_config.hidden_size
    n_layers = target_config.num_hidden_layers
    vocab = target_config.vocab_size
    inter = getattr(target_config, "intermediate_size", h * 4)
    target_params = vocab * h * 2 + n_layers * (h * h * 4 + h * inter * 3)
print(f"   Parameters: ~{target_params/1e9:.1f}B (estimated)")

In [ ]:
# Build EAGLE3 model
eagle3_model = OnlineEagle3Model(
    draft_model=draft_model,
    length=config["ttt_length"],
    attention_backend=config["attention_backend"],
).to(config["device"]).eval()

print(f"✅ EAGLE3 model ready (TTT length={config['ttt_length']})")

## 8. Run Evaluation on Sample

Let's evaluate on our first sample:

In [ ]:
# Move tensors to device
input_ids = input_ids.to(config["device"])
attention_mask = attention_mask.to(config["device"])
loss_mask = loss_mask.to(config["device"])

print(f"Running evaluation on sample...")
print(f"  Input shape: {input_ids.shape}")
print(f"  Device: {config['device']}")

start_time = time.time()

with torch.no_grad():
    # Get target model data
    eagle3_data = target_model.generate_eagle3_data(
        input_ids=input_ids,
        attention_mask=attention_mask,
        loss_mask=loss_mask,
    )
    
    print(f"\nEAGLE3 data generated:")
    print(f"  Input IDs shape: {eagle3_data.input_ids.shape}")
    print(f"  Hidden states shape: {eagle3_data.hidden_states.shape}")
    print(f"  Target shape: {eagle3_data.target.shape}")
    print(f"  Loss mask shape: {eagle3_data.loss_mask.shape}")
    
    # Run EAGLE3 forward pass
    plosses, _, acces = eagle3_model(
        input_ids=eagle3_data.input_ids,
        attention_mask=eagle3_data.attention_mask,
        loss_mask=eagle3_data.loss_mask,
        target=eagle3_data.target,
        hidden_states=eagle3_data.hidden_states,
    )

elapsed = time.time() - start_time

print(f"\n✅ Evaluation complete in {elapsed:.2f}s")
print(f"\nResults:")
print(f"  Number of TTT positions: {len(plosses)}")
for i, (loss, acc) in enumerate(zip(plosses, acces)):
    print(f"  Position {i}: Loss={loss.item():.4f}, Accuracy={acc.item():.1%}")

## 9. Detailed Prediction Analysis

Let's see what the draft model predicted vs what the target model said:

In [ ]:
def get_draft_predictions(eagle3_model, eagle3_data, ttt_length, attention_backend):
    """Get draft model predictions for each TTT position."""
    draft_model = eagle3_model.draft_model
    input_ids = eagle3_data.input_ids
    attention_mask = eagle3_data.attention_mask
    hidden_states = eagle3_data.hidden_states
    
    batch_size, seq_length = input_ids.shape
    device = hidden_states.device
    
    # Project hidden states
    hidden_states = draft_model.project_hidden_states(hidden_states)
    
    # Initialize cache
    if attention_backend == "sdpa" or attention_backend == "flex_attention_mla":
        cache_hidden = [[], []]
        past_key_values = None
    elif attention_backend == "flex_attention":
        cache_hidden = None
        past_key_values = DynamicCache()
    
    # Store predictions for each TTT position
    all_predictions = []
    
    # Initialize position_ids
    past_key_values_length = 0
    position_ids = torch.arange(
        past_key_values_length,
        seq_length + past_key_values_length,
        dtype=torch.long,
        device=device,
    )
    position_ids = position_ids.unsqueeze(0).view(-1, seq_length)
    
    for idx in range(ttt_length):
        # Embed input ids
        inputs_embeds = draft_model.embed_input_ids(input_ids)
        inputs_embeds = inputs_embeds.to(hidden_states.dtype)
        
        # Run backbone
        hidden_states_out = draft_model.backbone(
            input_embeds=inputs_embeds,
            hidden_states=hidden_states,
            cache_hidden=cache_hidden,
            attention_mask=attention_mask,
            position_ids=position_ids,
            past_key_values=past_key_values,
            use_cache=True,
        )
        
        hidden_states = hidden_states_out
        
        # Get logits and predictions
        logits = draft_model.compute_logits(hidden_states)
        predictions = logits.argmax(dim=-1)  # [batch, seq_len]
        all_predictions.append(predictions)
        
        # Pad for next iteration (except last)
        if idx < ttt_length - 1:
            input_ids = padding(input_ids, left=False)
            position_ids = padding(position_ids, left=False)
    
    return all_predictions

# Get draft predictions
print("Getting draft predictions...")
draft_predictions = get_draft_predictions(
    eagle3_model, 
    eagle3_data, 
    config["ttt_length"],
    config["attention_backend"]
)
print(f"✅ Got predictions for {len(draft_predictions)} TTT positions")

In [ ]:
# Visualize predictions
target_tokens = eagle3_data.target.argmax(dim=-1)[0]  # [seq_len]
input_tokens = eagle3_data.input_ids[0].tolist()  # [seq_len]

num_positions_to_show = min(config["show_first_n_positions"], len(input_tokens) - config["ttt_length"])

print(f"\n📊 Detailed Predictions (first {num_positions_to_show} positions)")
print(f"\nLegend: TTT=Time-to-Target offset")
print(f"  TTT 0 = predicting next token (position i predicts i+0)")
print(f"  TTT 1 = predicting one ahead (position i predicts i+1)")
print(f"  etc.\n")

print(f"{'Pos':<4} {'TTT':<4} {'Context (last 2 tokens)':<35} {'Draft Token':<20} {'Target Token':<20} {'Match'}")
print("-" * 110)

for pos_idx in range(num_positions_to_show):
    # Show context
    context_start = max(0, pos_idx - 1)
    context_tokens = input_tokens[context_start:pos_idx+1]
    context_text = tokenizer.decode(context_tokens, skip_special_tokens=False)
    context_text = repr(context_text)[1:-1]  # Escape
    context_text = ("..." + context_text[-32:]) if len(context_text) > 35 else context_text
    
    # For each TTT position
    for ttt_idx in range(config["ttt_length"]):
        target_pos = pos_idx + ttt_idx
        
        if target_pos >= len(target_tokens):
            break
        
        # Get predictions
        draft_pred = draft_predictions[ttt_idx][0, pos_idx].item()
        target_token = target_tokens[target_pos].item()
        
        match = "✓" if draft_pred == target_token else "✗"
        
        # Decode
        draft_text = tokenizer.decode([draft_pred], skip_special_tokens=False)
        target_text = tokenizer.decode([target_token], skip_special_tokens=False)
        
        draft_text = repr(draft_text)[1:-1][:18]
        target_text = repr(target_text)[1:-1][:18]
        
        # Only show context for first TTT position
        if ttt_idx == 0:
            print(f"{pos_idx:<4} {ttt_idx:<4} {context_text:<35} {draft_text:<20} {target_text:<20} {match}")
        else:
            print(f"{'':4} {ttt_idx:<4} {'':35} {draft_text:<20} {target_text:<20} {match}")
    
    # Separator every 3 positions
    if pos_idx < num_positions_to_show - 1 and pos_idx % 3 == 2:
        print()

## 10. Summary Statistics

In [ ]:
print("\n" + "=" * 80)
print("EVALUATION SUMMARY")
print("=" * 80)

mean_acc = sum(acc.item() for acc in acces) / len(acces)
mean_loss = sum(loss.item() for loss in plosses) / len(plosses)

print(f"\nPer-Position Results:")
for i, (acc, loss) in enumerate(zip(acces, plosses)):
    bar = "█" * int(acc.item() * 30) + "░" * (30 - int(acc.item() * 30))
    print(f"  TTT Position {i}: {bar} {acc.item():.1%}  (loss: {loss.item():.4f})")

print(f"\nOverall:")
print(f"  Mean Accuracy: {mean_acc:.1%}")
print(f"  Mean Loss:     {mean_loss:.4f}")
print(f"  Time:          {elapsed:.2f}s")
print(f"  Tokens:        {input_ids.shape[1]}")
print(f"  Throughput:    {input_ids.shape[1] / elapsed:.1f} tokens/s")

# Quality interpretation
print(f"\nQuality Assessment:")
if mean_acc >= 0.7:
    print("  ✅ EXCELLENT - Expected speedup: 2.5x - 3.5x")
elif mean_acc >= 0.5:
    print("  ⚠️  GOOD - Expected speedup: 1.5x - 2.5x")
elif mean_acc >= 0.3:
    print("  ⚠️  FAIR - Expected speedup: 1.2x - 1.5x")
else:
    print("  ❌ NEEDS WORK - Train longer or tune hyperparameters")

# Accuracy decay
if len(acces) > 1:
    decay = (acces[0].item() - acces[-1].item()) / acces[0].item() * 100
    print(f"\n  Accuracy decay (TTT 0→{len(acces)-1}): {decay:.1f}%")
    if decay > 30:
        print(f"    ⚠️  High decay - model struggles with longer predictions")
    else:
        print(f"    ✅ Low decay - consistent predictions across TTT positions")

## 11. Run on Multiple Samples (Optional)

Now let's run on all samples:

In [ ]:
from tqdm.notebook import tqdm

all_accs = [[] for _ in range(config["ttt_length"])]
all_losses = [[] for _ in range(config["ttt_length"])]
total_tokens = 0
total_time = 0.0

for i in tqdm(range(num_samples), desc="Evaluating samples"):
    prompt = TEST_PROMPTS[i]
    response = TEST_RESPONSES[i]
    
    # Build conversation
    messages = [{"role": "user", "content": prompt}]
    if template.system_prompt:
        messages = [{"role": "system", "content": template.system_prompt}] + messages
    messages.append({"role": "assistant", "content": response})
    
    # Tokenize
    try:
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    except Exception:
        text = f"{template.user_header}{prompt}{template.end_of_turn_token}{template.assistant_header}{response}{template.end_of_turn_token}"
    
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        return_offsets_mapping=True,
        max_length=config["max_length"], 
        truncation=True,
        add_special_tokens=False,
    )
    input_ids = inputs["input_ids"].to(config["device"])
    attention_mask = inputs["attention_mask"].to(config["device"])
    offsets = inputs["offset_mapping"][0]
    
    # Build loss mask
    loss_mask = torch.zeros(input_ids.shape[1], dtype=torch.long, device=config["device"])
    for match in re.finditer(assistant_pattern, text, re.DOTALL):
        start_char = match.start(1)
        end_char = match.end(1)
        for idx, (tok_start, tok_end) in enumerate(offsets):
            if tok_end > start_char and tok_start <= end_char:
                loss_mask[idx] = 1
    loss_mask = loss_mask.unsqueeze(0)
    
    # Evaluate
    start = time.time()
    with torch.no_grad():
        eagle3_data = target_model.generate_eagle3_data(
            input_ids=input_ids,
            attention_mask=attention_mask,
            loss_mask=loss_mask,
        )
        
        plosses, _, acces = eagle3_model(
            input_ids=eagle3_data.input_ids,
            attention_mask=eagle3_data.attention_mask,
            loss_mask=eagle3_data.loss_mask,
            target=eagle3_data.target,
            hidden_states=eagle3_data.hidden_states,
        )
    
    elapsed = time.time() - start
    total_time += elapsed
    total_tokens += input_ids.shape[1]
    
    # Collect metrics
    for j, (acc, loss) in enumerate(zip(acces, plosses)):
        all_accs[j].append(acc.item())
        all_losses[j].append(loss.item())
    
    mean_acc = sum(a for a in acces) / len(acces)
    print(f"  Sample {i+1}: acc={mean_acc.item():.1%} ({elapsed:.2f}s)")

print("\n✅ Evaluation complete!")

In [ ]:
# Final results
per_pos_acc = [sum(accs) / len(accs) for accs in all_accs]
per_pos_loss = [sum(losses) / len(losses) for losses in all_losses]
mean_acc = sum(per_pos_acc) / len(per_pos_acc)
mean_loss = sum(per_pos_loss) / len(per_pos_loss)

print("\n" + "=" * 80)
print(f"FINAL RESULTS ({num_samples} samples)")
print("=" * 80)

print("\nPer-Position Accuracy:")
for i, (acc, loss) in enumerate(zip(per_pos_acc, per_pos_loss)):
    bar = "█" * int(acc * 40) + "░" * (40 - int(acc * 40))
    print(f"  Position {i}: {bar} {acc:.1%}  (loss: {loss:.4f})")

print("\nSummary:")
print(f"  Mean Accuracy:    {mean_acc:.1%}")
print(f"  Mean Loss:        {mean_loss:.4f}")
print(f"  Total Time:       {total_time:.2f}s")
print(f"  Tokens Processed: {total_tokens:,}")
print(f"  Throughput:       {total_tokens / total_time:.1f} tokens/s")

# Visualization
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy plot
ax1.plot(range(len(per_pos_acc)), per_pos_acc, marker='o', linewidth=2, markersize=8)
ax1.set_xlabel('TTT Position', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Draft Model Accuracy by TTT Position', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])
ax1.axhline(y=mean_acc, color='r', linestyle='--', label=f'Mean: {mean_acc:.1%}')
ax1.legend()

# Loss plot
ax2.plot(range(len(per_pos_loss)), per_pos_loss, marker='s', linewidth=2, markersize=8, color='orange')
ax2.set_xlabel('TTT Position', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.set_title('Draft Model Loss by TTT Position', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=mean_loss, color='r', linestyle='--', label=f'Mean: {mean_loss:.4f}')
ax2.legend()

plt.tight_layout()
plt.show()

## Next Steps

Now that you understand the evaluation process:

1. **Modify the test samples** to use your own prompts/responses
2. **Try different chat templates** if using a different model
3. **Adjust TTT length** to see how performance changes
4. **Run on more samples** for more robust statistics
5. **Compare different checkpoints** from your training run

Key insights:
- **Loss mask** determines which tokens contribute to loss (only assistant responses)
- **TTT predictions** show how far ahead the draft model can predict accurately
- **Accuracy decay** indicates if the model struggles with longer-term predictions
- **High accuracy (>60%)** suggests good speculative decoding speedup potential